# Automate your network with Nornir – Python automation framework!

### Building network diagram

In [2]:
import os
from pprint import pprint
from colorama import Fore
import time

from nornir import InitNornir
os.chdir('..')

with InitNornir(config_file="scripts/config.yaml") as nr:
    



{'R1': Host: R1,
 'R10': Host: R10,
 'R2': Host: R2,
 'R3': Host: R3,
 'R4': Host: R4,
 'R5': Host: R5,
 'R6': Host: R6,
 'R7': Host: R7,
 'R8': Host: R8,
 'R9': Host: R9}


In [ ]:
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import netmiko_send_command

london_devices = nr.filter(F(groups__contains="London"))
result = london_devices.run(task=netmiko_send_command, command_string="show ip route")
print_result(result)

### Custom tasks

In [ ]:
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import netmiko_send_command

def get_commands(task, commands):
    for command in commands:
        task.run(task=netmiko_send_command, command_string=command)
        
london_devices = nr.filter(F(groups__contains="London"))
result = london_devices.run(task=get_commands, commands=["show ip int br", "show arp"])
print_result(result)
nr.close_connections()

## Building network diagram with Nornir

In [ ]:
%matplotlib inline

from typing import List, Dict, Tuple
import time

from colorama import Fore
from nornir import InitNornir
import networkx as nx
import matplotlib.pyplot as plt

from topology import parse_cdp_neighbors, build_graph

TOPOLOGY_FILENAME = "topology.png"

def draw_and_save_topology(graph: nx.Graph, edge_labels: List[Dict[Tuple[str, str], str]]) -> None:
    plt.figure(1, figsize=(12, 12))
    pos = nx.spring_layout(graph, seed=5)
    nx.draw_networkx(graph, pos, node_size=1300, node_color='orange')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels[0], label_pos=0.8)
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels[1], label_pos=0.2)
    plt.savefig(TOPOLOGY_FILENAME)
    print(f"The network topology diagram has been saved to {TOPOLOGY_FILENAME}")

start_time = time.time()
nr = InitNornir("config.yaml")
nr.run(task=parse_cdp_neighbors)
print("CDP details were successfully fetched using RESTCONF")
milestone = time.time()
time_to_run = milestone - start_time
print(f"{Fore.RED}It took {time_to_run:.2f} seconds to get and parse CDP details{Fore.RESET}")
graph, edge_labels = build_graph(nr.inventory.hosts.values())
draw_and_save_topology(graph, edge_labels)
time_to_run = time.time() - milestone
print(f"{Fore.RED}It took additional {time_to_run:.2f} seconds to draw and save the network topology{Fore.RESET}")

## END